In [10]:
#LIBRARIES
import pandas as pd
import numpy as np
import quandl
from dotenv import load_dotenv
import os

In [17]:
load_dotenv('.env')
quandl_key = os.environ.get('quandl_key')
print(quandl_key)


#quandl_key = 

None
